## Getting resources

We use [xp5k](https://github.com/pmorillon/xp5k) to manage the interaction with the Grid resources.

We create a new ```XP``` and define a new job.

Note that ```resources``` are the same as those given in a ```oarsub``` command line.

In [1]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
    :resources => ['nodes=10, walltime=1:00:00'],
    :site      => 'lyon',
    :types     => ["allow_classic_ssh"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 10})],
    :command   => "sleep 86400"
    })

## Submission

```submit``` is asynchronous, ```wait_for_jobs``` will wait for all the resources to be ready.

In [2]:
myxp.submit
myxp.wait_for_jobs

#<Proc:0x007feec7a02068@/Users/msimonin/msimonin@github.com/xp5k/lib/xp5k/xp.rb:118>

## Run command

```xp5k/rake``` include some helpers to run commands on the nodes of your reservation.

In [3]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'msimonin', :capture => true} do 
    'uptime -s'
end

Connected to sagittaire-50.lyon.grid5000.fr...
Connected to sagittaire-53.lyon.grid5000.fr...
Connected to sagittaire-55.lyon.grid5000.fr...
Connected to sagittaire-52.lyon.grid5000.fr...
Connected to sagittaire-58.lyon.grid5000.fr...
Connected to sagittaire-56.lyon.grid5000.fr...
Connected to sagittaire-9.lyon.grid5000.fr...
[command][sagittaire-50.lyon.grid5000.fr] uptime -s
[command][sagittaire-52.lyon.grid5000.fr] uptime -s
[command][sagittaire-55.lyon.grid5000.fr] uptime -s
[command][sagittaire-58.lyon.grid5000.fr] uptime -s
[command][sagittaire-59.lyon.grid5000.fr] uptime -s
[command][sagittaire-9.lyon.grid5000.fr] uptime -s
---- sagittaire-50.lyon.grid5000.fr ------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2016-01-10 23:50:35

---- sagittaire-52.lyon.grid5000.fr ---------------------------------------------------------------------------------------------------

{"sagittaire-52.lyon.grid5000.fr"=>["2016-01-10 23:39:09"], "sagittaire-50.lyon.grid5000.fr"=>["2016-01-10 23:50:35"], "sagittaire-53.lyon.grid5000.fr"=>["2016-01-10 23:50:35"], "sagittaire-56.lyon.grid5000.fr"=>["2016-01-10 23:50:37"], "sagittaire-54.lyon.grid5000.fr"=>["2016-01-10 23:50:36"], "sagittaire-55.lyon.grid5000.fr"=>["2016-01-10 23:52:11"], "sagittaire-58.lyon.grid5000.fr"=>["2016-01-10 23:50:36"], "sagittaire-57.lyon.grid5000.fr"=>["2016-01-10 18:29:52"], "sagittaire-59.lyon.grid5000.fr"=>["2016-01-10 23:50:37"], "sagittaire-9.lyon.grid5000.fr"=>["2016-01-10 23:50:32"]}

## Draw the bar plot

The output of the above command is a map :

 * each key is a server name 
 * a value is the outputs of the commands run on this server
 
e.g : ``` 'sagittaire-1.lyon.grid5000.fr' => ['out1', ... , 'outn'] ```

The following code compute the time since last reboot in minute.

In [4]:
require 'nyaplot'
require 'time'

now = Time.now.to_i
plot = Nyaplot::Plot.new
df = Nyaplot::DataFrame.new({
    host: h.keys
           .map{|x| x.split('.').first},
    uptime: h.values
             .map{|x| x.first}
    .map{|x| ((now - Time.parse(x).to_f)/60).round(2)}
    })
    

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t    console.log('Finished loading Nyaplotjs');\n\n\t};\n\n\n});});\n}\n"

host,uptime
sagittaire-52,37.85
sagittaire-50,26.42
sagittaire-53,26.42
sagittaire-56,26.38
sagittaire-54,26.4
sagittaire-55,24.82
sagittaire-58,26.4
sagittaire-57,347.13
sagittaire-59,26.38
sagittaire-9,26.47


In [5]:
plot=Nyaplot::Plot.new
plot.add_with_df(df, :bar, :host, :uptime)
plot.configure do
    x_label('')
    y_label('Uptime (ms)')
    rotate_x_label(-90)
end


#<Nyaplot::Plot:0x007feec47e0288 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007feec47e00a8 @properties={:type=>:bar, :options=>{:x=>:host, :y=>:uptime}, :data=>"3b3ad416-2683-4133-ae97-05cb574c5d28"}, @xrange=["sagittaire-52", "sagittaire-50", "sagittaire-53", "sagittaire-56", "sagittaire-54", "sagittaire-55", "sagittaire-58", "sagittaire-57", "sagittaire-59", "sagittaire-9"], @yrange=[0, 347.13]>], :options=>{:x_label=>"", :y_label=>"Uptime (ms)", :rotate_x_label=>-90, :width=>700, :xrange=>["sagittaire-52", "sagittaire-50", "sagittaire-53", "sagittaire-56", "sagittaire-54", "sagittaire-55", "sagittaire-58", "sagittaire-57", "sagittaire-59", "sagittaire-9"], :yrange=>[0, 347.13]}}>

In [ ]:
myxp.clean